In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



df = pd.read_csv('combined_dataset.csv')

# Convert the index to datetime
df[df.columns[0]] = pd.to_datetime(df[df.columns[0]])
df.set_index(df.columns[0], inplace=True)

# Remove duplicate columns if any
duplicate_columns = df.columns[df.columns.duplicated()]
df = df.drop(columns=duplicate_columns)

# Split the data based on the index
split_date = pd.Timestamp('2023-10-01')
train = df.loc[df.index < split_date]
test = df.loc[df.index >= split_date]

# Define the features and target variable for training
X_train = train.drop(columns=['NASDAQ_Volatility']).values
y_train = train['NASDAQ_Volatility'].values

# Define the features and target variable for testing
X_test = test.drop(columns=['NASDAQ_Volatility']).values
y_test = test['NASDAQ_Volatility'].values

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape input to be 3D for LSTM [samples, timesteps, features]
n_timesteps = X_train.shape[1]  # This should be the number of features
X_train = X_train.reshape((X_train.shape[0], 1, n_timesteps))
X_test = X_test.reshape((X_test.shape[0], 1, n_timesteps))

# Build LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, n_timesteps)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Predict and evaluate
y_pred = model.predict(X_test)
r_squared = r2_score(y_test, y_pred)
print(f'R-squared on Test Set: {r_squared}')


Epoch 1/50
531/531 [==============================] - 1s 678us/step - loss: 0.0066
Epoch 2/50
531/531 [==============================] - 0s 641us/step - loss: 0.0015
Epoch 3/50
531/531 [==============================] - 0s 637us/step - loss: 0.0018
Epoch 4/50
531/531 [==============================] - 0s 632us/step - loss: 6.2051e-04
Epoch 5/50
531/531 [==============================] - 0s 637us/step - loss: 2.6071e-04
Epoch 6/50
531/531 [==============================] - 0s 629us/step - loss: 1.6927e-04
Epoch 7/50
531/531 [==============================] - 0s 614us/step - loss: 1.2204e-04
Epoch 8/50
531/531 [==============================] - 0s 628us/step - loss: 1.2131e-04
Epoch 9/50
531/531 [==============================] - 0s 619us/step - loss: 1.0712e-04
Epoch 10/50
531/531 [==============================] - 0s 622us/step - loss: 1.8457e-04
Epoch 11/50
531/531 [==============================] - 0s 657us/step - loss: 2.5597e-04
Epoch 12/50
531/531 [==============================] 

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Dropout, Bidirectional

import os
import random
import numpy as np
import tensorflow as tf

def custom_set_seed(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    
custom_set_seed(42)

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv('combined_dataset.csv')

# Set the index to the first column
df.set_index(df.columns[0], inplace=True)

# Remove duplicate columns if any
duplicate_columns = df.columns[df.columns.duplicated()]
df = df.drop(columns=duplicate_columns)

# Define features (X) and target (y)
X = df.drop(columns=['NASDAQ_Volatility']).values
y = df['NASDAQ_Volatility'].values

# Change the splitting of the data to use a date threshold
# Convert index to datetime
df.index = pd.to_datetime(df.index)

# Define the split date
split_date = pd.Timestamp('2023-10-01')

# Split the data into train and test sets
train = df.loc[df.index < split_date]
test = df.loc[df.index >= split_date]

X_train, y_train = train.drop(columns=['NASDAQ_Volatility']).values, train['NASDAQ_Volatility'].values
X_test, y_test = test.drop(columns=['NASDAQ_Volatility']).values, test['NASDAQ_Volatility'].values

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for LSTM layer
n_timesteps = 1 
X_train = X_train.reshape((X_train.shape[0], n_timesteps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], n_timesteps, X_test.shape[1]))

# Build the model
model = Sequential()
model.add(Bidirectional(LSTM(150, activation='relu', return_sequences=True), input_shape=(n_timesteps, X_train.shape[2])))
model.add(LSTM(78, activation='relu', return_sequences=False))#58
model.add(Dropout(0.20))
model.add(Dense(30, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=1)#32

# Predict using the test set
y_pred = model.predict(X_test)

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
r_squared


Epoch 1/10
340/340 [==============================] - 2s 3ms/step - loss: 1.8404e-05
Epoch 2/10
340/340 [==============================] - 1s 3ms/step - loss: 8.9648e-05
Epoch 3/10
340/340 [==============================] - 1s 4ms/step - loss: 3.2349e-05
Epoch 4/10
340/340 [==============================] - 1s 3ms/step - loss: 1.2292e-05
Epoch 5/10
340/340 [==============================] - 1s 3ms/step - loss: 4.8124e-05
Epoch 6/10
340/340 [==============================] - 1s 3ms/step - loss: 2.1114e-05
Epoch 7/10
340/340 [==============================] - 1s 3ms/step - loss: 9.2558e-06
Epoch 8/10
340/340 [==============================] - 1s 3ms/step - loss: 2.6990e-06
Epoch 9/10
340/340 [==============================] - 1s 3ms/step - loss: 4.6689e-06
Epoch 10/10
69/69 [==============================] - 0s 824us/step


-0.21375548694351632